# スタッキング001

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
path = './home-credit-default-risk/'

In [3]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Lgb, Model1Xgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



## 共通関数定義

In [4]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [5]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [6]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [7]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [8]:
### bureau_balance
bureau = FeatureEngineering.bureau_and_balance(path)
bureau.head()

Memory usage of dataframe: 4114.69 MB
Memory usage after optimization: 1189.22 MB
Decreased by 71.1%


,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [9]:
bureau = bureau.replace([np.inf, -np.inf], np.nan)
###bureau = bureau.fillna(bureau.max() + (bureau.max() * 0.4))
bureau.head()

,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [10]:
### previos_application
previous = FeatureEngineering.previous_application(path)
previous.head()

Memory usage of dataframe: 2434.34 MB
Memory usage after optimization: 818.55 MB
Decreased by 66.4%


,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [11]:
previous = previous.replace([np.inf, -np.inf], np.nan)
###previous = previous.fillna(previous.max() + (previous.max() * 0.4))
previous.head()

,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [12]:
### pos_cash
pos = FeatureEngineering.pos_cash(path)
pos.head()

Memory usage of dataframe: 147.95 MB
Memory usage after optimization: 46.96 MB
Decreased by 68.3%


,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [13]:
pos = pos.replace([np.inf, -np.inf], np.nan)
###pos = pos.fillna(pos.max() + (pos.max() * 0.4))
pos.head()

,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [14]:
### instllment_payment
installments = FeatureEngineering.installments_payments(path)
installments.head()

Memory usage of dataframe: 199.49 MB
Memory usage after optimization: 73.84 MB
Decreased by 63.0%


,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [15]:
installments = installments.replace([np.inf, -np.inf], np.nan)
###installments = installments.fillna(installments.max() + (installments.max() * 0.4))
installments.head()

,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [16]:
### credit_card_balance
credit = FeatureEngineering.credit_card_balance(path)
credit.head()

Memory usage of dataframe: 173.03 MB
Memory usage after optimization: 66.37 MB
Decreased by 61.6%


,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [17]:
credit = credit.replace([np.inf, -np.inf], np.nan)
###credit = credit.fillna(credit.max() + (credit.max() * 0.4))
credit.head()

,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [18]:
### application
df = FeatureEngineering.application_train_test(path, 'application_train.csv')
df.head()

Memory usage of dataframe: 404.30 MB
Memory usage after optimization: 169.87 MB
Decreased by 58.0%


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


In [19]:
df = df.replace([np.inf, -np.inf], np.nan)
#df = df.fillna(installments.max() + (installments.max() * 0.4))
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


### データフレームを結合

In [20]:
### データフレームをマージ
df = pd.merge(df, bureau, on='SK_ID_CURR', how='left')
df = pd.merge(df, installments, on='SK_ID_CURR', how='left')
df = pd.merge(df, previous, on='SK_ID_CURR', how='left')
df = pd.merge(df, credit, on='SK_ID_CURR', how='left')
df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


In [21]:
#df = FeatureEngineering.clean_data(df)
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


In [22]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 3396 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 3279 features are remained after removing non-informative features
1142 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2137 features are remained after removing features not interesting for LightGBM classifier
---=> 90 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2227 features

---=> df final shape: (356249, 2227)  <=--- 



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 2227 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 2223 features are remained after removing non-informative features
111 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2112 features are remained after removing features not interesting for LightGBM classifier
---=> 31 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2143 features

---=> df final shape: (356249, 2143)  <=--- 



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,APPROVED_NAME_GOODS_CATEGORY_Insurance_MEAN_high_risk,APPROVED_NAME_GOODS_CATEGORY_Insurance_MEAN_low_risk,REFUSED_prev_missing_MIN_high_risk,REFUSED_prev_missing_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1,0,1,0,0,1,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,1,0,0,1,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,0,1,0,0,1,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,1,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,0,1,0,0,1,0,0,0,0


In [24]:
### 変数選択003
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 2143 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 2141 features are remained after removing non-informative features
53 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2088 features are remained after removing features not interesting for LightGBM classifier
---=> 24 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2112 features

---=> df final shape: (356249, 2112)  <=--- 



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,1,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,1,0,1,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,1,0,1,0,0,0,0,0,0


In [25]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,1,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,1,0,1,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,1,0,1,0,0,0,0,0,0


In [26]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356249, 2112)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,1,0,1,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,1,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,1,1,0,1,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,0,1,0,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,1,1,0,1,0,0,0,0,0,0


In [27]:
### split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 2112)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,ACTIVE_Last_status_0_MEAN_SUM_low_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk
307505,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,1,0,0,0,0,0,0
307506,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,1,0,0,0,0,0,0
307507,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,1,0,1,0,1,0
307508,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,0,1,1,0,1,0,1,0
307509,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,1,0,0,0,0,0,0


In [28]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.044426
1,100005,0.134157
2,100013,0.041392
3,100028,0.051442
4,100038,0.193094


In [29]:
app_train = app_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_high_risk,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,1,0,0,0,0,0,0,0.044426
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,0,1,0,0,0,0,0,0,0.134157
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,1,0,1,0,1,0,0.041392
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,1,0,1,0,1,0,0.051442
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,1,0,0,0,0,0,0,0.193094


In [30]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [31]:
app_train = app_train.merge(train_set2a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,APPROVED_NAME_CASH_LOAN_PURPOSE_Buying_a_home_MEAN_low_risk,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,0,0,0,0,0,0.044426,0.025023
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,0,0,0,0,0,0.134157,0.090335
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,1,0,1,0,1,0,0.041392,0.037033
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,1,0,1,0,1,0,0.051442,0.036644
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,0,0,0,0,0,0.193094,0.172674


In [32]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set2xgb.head()

,SK_ID_CURR,set2x
0,100001,0.035517
1,100005,0.105594
2,100013,0.043767
3,100028,0.034733
4,100038,0.156875


In [33]:
app_train = app_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_prev_missing_MIN_low_risk,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,0,0,0,0.044426,0.025023,0.035517
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,0,0,0,0,0,0,0.134157,0.090335,0.105594
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.041392,0.037033,0.043767
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.051442,0.036644,0.034733
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,0,0.193094,0.172674,0.156875


In [34]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [35]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0.044426,0.025023,0.035517,0.049120
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0.134157,0.090335,0.105594,0.115729
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,0,0.041392,0.037033,0.043767,0.064166
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,0,0.051442,0.036644,0.034733,0.038931
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0.193094,0.172674,0.156875,0.195575


In [36]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0,0,0.044426,0.025023,0.035517,0.049120
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0,0,0.134157,0.090335,0.105594,0.115729
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,1,0,0.041392,0.037033,0.043767,0.064166
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,1,0,0.051442,0.036644,0.034733,0.038931
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0,0,0.193094,0.172674,0.156875,0.195575


In [37]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0.044426,0.025023,0.035517,0.049120
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0.134157,0.090335,0.105594,0.115729
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,0,0.041392,0.037033,0.043767,0.064166
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,0,0.051442,0.036644,0.034733,0.038931
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0.193094,0.172674,0.156875,0.195575


In [38]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0.044426,0.025023,0.035517,0.049120
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0.134157,0.090335,0.105594,0.115729
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,0,0.041392,0.037033,0.043767,0.064166
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,0,0.051442,0.036644,0.034733,0.038931
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0.193094,0.172674,0.156875,0.195575


### modelを生成

In [39]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [40]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [41]:
# infをnanへ置換
#app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0.268900,0.156048,0.234038,0.223662
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0.010106,0.012034,0.015184,0.024137
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0.068937,0.026333,0.046318,0.035314
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0.007476,0.029856,0.026462,0.031135
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0.080712,0.034698,0.034950,0.035167


In [42]:
# 欠損値補完
app_test = app_test.replace([np.inf, -np.inf], np.nan)
#app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0.044426,0.025023,0.035517,0.049120
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0.134157,0.090335,0.105594,0.115729
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,0,0.041392,0.037033,0.043767,0.064166
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,0,0.051442,0.036644,0.034733,0.038931
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0.193094,0.172674,0.156875,0.195575


In [43]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [44]:
### indexを設定
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018845,...,0,0,0,0,0,0,0.044426,0.025023,0.035517,0.049120
100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035797,...,0,0,0,0,0,0,0.134157,0.090335,0.105594,0.115729
100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019104,...,1,0,1,0,1,0,0.041392,0.037033,0.043767,0.064166
100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,...,1,0,1,0,1,0,0.051442,0.036644,0.034733,0.038931
100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010033,...,0,0,0,0,0,0,0.193094,0.172674,0.156875,0.195575


In [45]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,0.720375,-0.717914,-0.664541,-0.577536,-0.307107,-0.075103,-0.451788,-0.239271,-0.146484,...,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,-0.414015,-0.522813,-0.452849,-0.328311
100005,NaN,-1.388165,-0.717914,-0.664541,-0.577536,-0.637955,-0.934824,-0.671917,-0.970093,1.079102,...,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,0.467630,0.166746,0.283075,0.365053
100013,NaN,-1.388165,1.392925,-0.664541,-0.577536,0.313233,0.159594,2.943901,0.247945,-0.127563,...,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.443824,-0.396011,-0.366214,-0.171692
100028,NaN,0.720375,-0.717914,-0.664541,2.192089,1.347132,2.424815,1.511670,2.805824,0.399658,...,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.345078,-0.400115,-0.461078,-0.434372
100038,NaN,-1.388165,1.392925,1.504799,0.807276,0.106453,0.065769,0.342102,0.235764,-0.783203,...,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,1.046702,1.036070,0.821604,1.196217


In [46]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100001,NaN,0.720375,-0.717914,-0.664541,-0.577536,-0.307107,-0.075103,-0.451788,-0.239271,...,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,-0.414015,-0.522813,-0.452849,-0.328311
1,100005,NaN,-1.388165,-0.717914,-0.664541,-0.577536,-0.637955,-0.934824,-0.671917,-0.970093,...,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,0.467630,0.166746,0.283075,0.365053
2,100013,NaN,-1.388165,1.392925,-0.664541,-0.577536,0.313233,0.159594,2.943901,0.247945,...,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.443824,-0.396011,-0.366214,-0.171692
3,100028,NaN,0.720375,-0.717914,-0.664541,2.192089,1.347132,2.424815,1.511670,2.805824,...,1.685051,-0.150514,2.007797,-0.302406,2.007367,-0.302336,-0.345078,-0.400115,-0.461078,-0.434372
4,100038,NaN,-1.388165,1.392925,1.504799,0.807276,0.106453,0.065769,0.342102,0.235764,...,-0.593454,-0.150514,-0.498058,-0.302406,-0.498165,-0.302336,1.046702,1.036070,0.821604,1.196217


In [47]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100002,1.0,-1.387695,-0.717773,-0.664551,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,1.791992,0.860352,1.631836,1.488281
1,100003,0.0,0.720215,-0.717773,1.504883,-0.577637,0.933594,1.725586,0.592773,1.599609,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,-0.750977,-0.660156,-0.666504,-0.588379
2,100004,0.0,-1.387695,1.392578,-0.664551,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,-0.173218,-0.508789,-0.339355,-0.471924
3,100006,0.0,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,...,1.684570,-0.150513,-0.498047,3.306641,-0.498047,3.306641,-0.776855,-0.471680,-0.547852,-0.515625
4,100007,0.0,-1.387695,-0.717773,-0.664551,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302246,-0.057495,-0.420654,-0.458740,-0.473633


In [48]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,set2nn,set2a,set2x,set3a
0,100001,NaN,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.075073,-0.451904,-0.239258,...,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.302246,-0.414062,-0.522949,-0.452881,-0.328369
1,100005,NaN,-1.387695,-0.717773,-0.664551,-0.577637,-0.638184,-0.935059,-0.671875,-0.970215,...,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.302246,0.467529,0.166748,0.282959,0.364990
2,100013,NaN,-1.387695,1.392578,-0.664551,-0.577637,0.313232,0.159546,2.943359,0.247925,...,1.684570,-0.150513,2.007812,-0.30249,2.007812,-0.302246,-0.443848,-0.395996,-0.366211,-0.171631
3,100028,NaN,0.720215,-0.717773,-0.664551,2.191406,1.346680,2.425781,1.511719,2.806641,...,1.684570,-0.150513,2.007812,-0.30249,2.007812,-0.302246,-0.344971,-0.400146,-0.461182,-0.434326
4,100038,NaN,-1.387695,1.392578,1.504883,0.807129,0.106445,0.065796,0.342041,0.235718,...,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.302246,1.046875,1.036133,0.821777,1.196289


In [49]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [50]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [51]:
### training Xgb
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, x_train, y_train, x_test)

[21:12:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "num_round", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.78464	eval-auc:0.78340
[1]	train-auc:0.79232	eval-auc:0.79075
[2]	train-auc:0.79420	eval-auc:0.79200
[3]	train-auc:0.79580	eval-auc:0.79368
[4]	train-auc:0.79650	eval-auc:0.79440
[5]	train-auc:0.79640	eval-auc:0.79433
[6]	train-auc:0.79679	eval-auc:0.79418
[7]	train-auc:0.79689	eval-auc:0.79426
[8]	train-auc:0.79728	eval-auc:0.79471
[9]	train-auc:0.79742	eval-auc:0.79483
[10]	train-auc:0.79765	eval-auc:0.79486
[11]	train-auc:0.79758	eval-auc:0.79486
[12]	train-auc:0.79765	eval-auc:0.79496
[13]	train-auc:0.79759	eval-auc:0.79495
[14

[192]	train-auc:0.81650	eval-auc:0.79658
[193]	train-auc:0.81660	eval-auc:0.79661
[194]	train-auc:0.81677	eval-auc:0.79660
[195]	train-auc:0.81687	eval-auc:0.79660
[196]	train-auc:0.81703	eval-auc:0.79659
[197]	train-auc:0.81722	eval-auc:0.79657
[198]	train-auc:0.81738	eval-auc:0.79656
[199]	train-auc:0.81753	eval-auc:0.79655
[200]	train-auc:0.81773	eval-auc:0.79655
[201]	train-auc:0.81791	eval-auc:0.79655
[202]	train-auc:0.81808	eval-auc:0.79654
[203]	train-auc:0.81826	eval-auc:0.79656
[204]	train-auc:0.81841	eval-auc:0.79657
[205]	train-auc:0.81857	eval-auc:0.79658
[206]	train-auc:0.81869	eval-auc:0.79659
[207]	train-auc:0.81881	eval-auc:0.79660
[208]	train-auc:0.81891	eval-auc:0.79661
[209]	train-auc:0.81910	eval-auc:0.79663
[210]	train-auc:0.81923	eval-auc:0.79666
[211]	train-auc:0.81937	eval-auc:0.79666
[212]	train-auc:0.81951	eval-auc:0.79664
[213]	train-auc:0.81976	eval-auc:0.79662
[214]	train-auc:0.81988	eval-auc:0.79666
[215]	train-auc:0.82014	eval-auc:0.79666
[216]	train-auc:

[392]	train-auc:0.85556	eval-auc:0.79638
[393]	train-auc:0.85578	eval-auc:0.79638
[394]	train-auc:0.85601	eval-auc:0.79637
[395]	train-auc:0.85613	eval-auc:0.79637
[396]	train-auc:0.85643	eval-auc:0.79638
[397]	train-auc:0.85661	eval-auc:0.79638
[398]	train-auc:0.85687	eval-auc:0.79637
[399]	train-auc:0.85711	eval-auc:0.79638
[400]	train-auc:0.85737	eval-auc:0.79638
[401]	train-auc:0.85757	eval-auc:0.79637
[402]	train-auc:0.85785	eval-auc:0.79638
[403]	train-auc:0.85812	eval-auc:0.79637
[404]	train-auc:0.85834	eval-auc:0.79635
[405]	train-auc:0.85853	eval-auc:0.79635
[406]	train-auc:0.85873	eval-auc:0.79635
[407]	train-auc:0.85901	eval-auc:0.79635
[408]	train-auc:0.85926	eval-auc:0.79634
[409]	train-auc:0.85944	eval-auc:0.79633
[410]	train-auc:0.85969	eval-auc:0.79630
[411]	train-auc:0.85985	eval-auc:0.79630
[412]	train-auc:0.86011	eval-auc:0.79627
[413]	train-auc:0.86031	eval-auc:0.79627
[414]	train-auc:0.86051	eval-auc:0.79629
[415]	train-auc:0.86066	eval-auc:0.79629
[416]	train-auc:

[592]	train-auc:0.89229	eval-auc:0.79607
[593]	train-auc:0.89248	eval-auc:0.79607
[594]	train-auc:0.89274	eval-auc:0.79604
[595]	train-auc:0.89283	eval-auc:0.79605
[596]	train-auc:0.89297	eval-auc:0.79603
[597]	train-auc:0.89307	eval-auc:0.79603
[598]	train-auc:0.89318	eval-auc:0.79603
[599]	train-auc:0.89328	eval-auc:0.79603
[600]	train-auc:0.89347	eval-auc:0.79602
[601]	train-auc:0.89365	eval-auc:0.79602
[602]	train-auc:0.89381	eval-auc:0.79602
[603]	train-auc:0.89397	eval-auc:0.79603
[604]	train-auc:0.89411	eval-auc:0.79604
[605]	train-auc:0.89423	eval-auc:0.79603
[606]	train-auc:0.89438	eval-auc:0.79602
[607]	train-auc:0.89451	eval-auc:0.79601
[608]	train-auc:0.89468	eval-auc:0.79600
[609]	train-auc:0.89487	eval-auc:0.79600
[610]	train-auc:0.89500	eval-auc:0.79599
[611]	train-auc:0.89517	eval-auc:0.79600
[612]	train-auc:0.89529	eval-auc:0.79599
[613]	train-auc:0.89547	eval-auc:0.79599
[614]	train-auc:0.89557	eval-auc:0.79596
[615]	train-auc:0.89571	eval-auc:0.79598
[616]	train-auc:

[792]	train-auc:0.92028	eval-auc:0.79532
[793]	train-auc:0.92039	eval-auc:0.79533
[794]	train-auc:0.92044	eval-auc:0.79533
[795]	train-auc:0.92057	eval-auc:0.79533
[796]	train-auc:0.92073	eval-auc:0.79533
[797]	train-auc:0.92089	eval-auc:0.79534
[798]	train-auc:0.92106	eval-auc:0.79534
[799]	train-auc:0.92116	eval-auc:0.79534
[800]	train-auc:0.92122	eval-auc:0.79534
[801]	train-auc:0.92136	eval-auc:0.79532
[802]	train-auc:0.92152	eval-auc:0.79533
[803]	train-auc:0.92167	eval-auc:0.79532
[804]	train-auc:0.92176	eval-auc:0.79533
[805]	train-auc:0.92183	eval-auc:0.79532
[806]	train-auc:0.92203	eval-auc:0.79530
[807]	train-auc:0.92219	eval-auc:0.79530
[808]	train-auc:0.92230	eval-auc:0.79530
[809]	train-auc:0.92247	eval-auc:0.79531
[810]	train-auc:0.92268	eval-auc:0.79531
[811]	train-auc:0.92276	eval-auc:0.79530
[812]	train-auc:0.92291	eval-auc:0.79530
[813]	train-auc:0.92305	eval-auc:0.79531
[814]	train-auc:0.92325	eval-auc:0.79529
[815]	train-auc:0.92343	eval-auc:0.79530
[816]	train-auc:

...

[162]	train-auc:0.81235	eval-auc:0.79429
[163]	train-auc:0.81255	eval-auc:0.79431
[164]	train-auc:0.81262	eval-auc:0.79430
[165]	train-auc:0.81275	eval-auc:0.79430
[166]	train-auc:0.81287	eval-auc:0.79433
[167]	train-auc:0.81296	eval-auc:0.79433
[168]	train-auc:0.81309	eval-auc:0.79433
[169]	train-auc:0.81328	eval-auc:0.79433
[170]	train-auc:0.81348	eval-auc:0.79436
[171]	train-auc:0.81356	eval-auc:0.79438
[172]	train-auc:0.81365	eval-auc:0.79438
[173]	train-auc:0.81375	eval-auc:0.79439
[174]	train-auc:0.81385	eval-auc:0.79438
[175]	train-auc:0.81394	eval-auc:0.79438
[176]	train-auc:0.81409	eval-auc:0.79438
[177]	train-auc:0.81424	eval-auc:0.79438
[178]	train-auc:0.81437	eval-auc:0.79436
[179]	train-auc:0.81457	eval-auc:0.79435
[180]	train-auc:0.81467	eval-auc:0.79437
[181]	train-auc:0.81482	eval-auc:0.79440
[182]	train-auc:0.81502	eval-auc:0.79441
[183]	train-auc:0.81519	eval-auc:0.79441
[184]	train-auc:0.81541	eval-auc:0.79442
[185]	train-auc:0.81558	eval-auc:0.79441
[186]	train-auc:

[362]	train-auc:0.84921	eval-auc:0.79410
[363]	train-auc:0.84948	eval-auc:0.79409
[364]	train-auc:0.84964	eval-auc:0.79408
[365]	train-auc:0.84987	eval-auc:0.79408
[366]	train-auc:0.85014	eval-auc:0.79407
[367]	train-auc:0.85035	eval-auc:0.79407
[368]	train-auc:0.85056	eval-auc:0.79407
[369]	train-auc:0.85076	eval-auc:0.79407
[370]	train-auc:0.85099	eval-auc:0.79406
[371]	train-auc:0.85124	eval-auc:0.79403
[372]	train-auc:0.85148	eval-auc:0.79402
[373]	train-auc:0.85175	eval-auc:0.79403
[374]	train-auc:0.85197	eval-auc:0.79404
[375]	train-auc:0.85213	eval-auc:0.79405
[376]	train-auc:0.85236	eval-auc:0.79403
[377]	train-auc:0.85247	eval-auc:0.79404
[378]	train-auc:0.85269	eval-auc:0.79404
[379]	train-auc:0.85296	eval-auc:0.79405
[380]	train-auc:0.85316	eval-auc:0.79402
[381]	train-auc:0.85338	eval-auc:0.79401
[382]	train-auc:0.85365	eval-auc:0.79402
[383]	train-auc:0.85384	eval-auc:0.79403
[384]	train-auc:0.85408	eval-auc:0.79404
[385]	train-auc:0.85432	eval-auc:0.79404
[386]	train-auc:

[562]	train-auc:0.88763	eval-auc:0.79339
[563]	train-auc:0.88777	eval-auc:0.79338
[564]	train-auc:0.88787	eval-auc:0.79337
[565]	train-auc:0.88804	eval-auc:0.79338
[566]	train-auc:0.88817	eval-auc:0.79336
[567]	train-auc:0.88834	eval-auc:0.79335
[568]	train-auc:0.88854	eval-auc:0.79333
[569]	train-auc:0.88874	eval-auc:0.79332
[570]	train-auc:0.88883	eval-auc:0.79332
[571]	train-auc:0.88897	eval-auc:0.79331
[572]	train-auc:0.88911	eval-auc:0.79331
[573]	train-auc:0.88928	eval-auc:0.79330
[574]	train-auc:0.88940	eval-auc:0.79328
[575]	train-auc:0.88948	eval-auc:0.79328
[576]	train-auc:0.88967	eval-auc:0.79326
[577]	train-auc:0.88984	eval-auc:0.79324
[578]	train-auc:0.88996	eval-auc:0.79325
[579]	train-auc:0.89004	eval-auc:0.79326
[580]	train-auc:0.89014	eval-auc:0.79326
[581]	train-auc:0.89030	eval-auc:0.79324
[582]	train-auc:0.89049	eval-auc:0.79323
[583]	train-auc:0.89061	eval-auc:0.79323
[584]	train-auc:0.89069	eval-auc:0.79324
[585]	train-auc:0.89089	eval-auc:0.79325
[586]	train-auc:

[762]	train-auc:0.91460	eval-auc:0.79258
[763]	train-auc:0.91476	eval-auc:0.79258
[764]	train-auc:0.91486	eval-auc:0.79257
[765]	train-auc:0.91503	eval-auc:0.79258
[766]	train-auc:0.91522	eval-auc:0.79255
[767]	train-auc:0.91534	eval-auc:0.79256
[768]	train-auc:0.91549	eval-auc:0.79255
[769]	train-auc:0.91564	eval-auc:0.79255
[770]	train-auc:0.91581	eval-auc:0.79254
[771]	train-auc:0.91599	eval-auc:0.79254
[772]	train-auc:0.91610	eval-auc:0.79253
[773]	train-auc:0.91629	eval-auc:0.79252
[774]	train-auc:0.91639	eval-auc:0.79251
[775]	train-auc:0.91647	eval-auc:0.79250
[776]	train-auc:0.91658	eval-auc:0.79249
[777]	train-auc:0.91672	eval-auc:0.79249
[778]	train-auc:0.91684	eval-auc:0.79250
[779]	train-auc:0.91701	eval-auc:0.79249
[780]	train-auc:0.91712	eval-auc:0.79249
[781]	train-auc:0.91730	eval-auc:0.79249
[782]	train-auc:0.91741	eval-auc:0.79250
[783]	train-auc:0.91758	eval-auc:0.79248
[784]	train-auc:0.91769	eval-auc:0.79247
[785]	train-auc:0.91777	eval-auc:0.79247
[786]	train-auc:

[962]	train-auc:0.93989	eval-auc:0.79199
[963]	train-auc:0.93999	eval-auc:0.79200
[964]	train-auc:0.94011	eval-auc:0.79200
[965]	train-auc:0.94024	eval-auc:0.79198
[966]	train-auc:0.94032	eval-auc:0.79199
[967]	train-auc:0.94037	eval-auc:0.79199
[968]	train-auc:0.94043	eval-auc:0.79197
[969]	train-auc:0.94051	eval-auc:0.79197
[970]	train-auc:0.94063	eval-auc:0.79195
[971]	train-auc:0.94078	eval-auc:0.79195
[972]	train-auc:0.94091	eval-auc:0.79195
[973]	train-auc:0.94103	eval-auc:0.79195
[974]	train-auc:0.94113	eval-auc:0.79197
[975]	train-auc:0.94125	eval-auc:0.79195
[976]	train-auc:0.94136	eval-auc:0.79195
[977]	train-auc:0.94153	eval-auc:0.79196
[978]	train-auc:0.94166	eval-auc:0.79197
[979]	train-auc:0.94179	eval-auc:0.79196
[980]	train-auc:0.94195	eval-auc:0.79196
[981]	train-auc:0.94206	eval-auc:0.79197
[982]	train-auc:0.94215	eval-auc:0.79196
[983]	train-auc:0.94229	eval-auc:0.79195
[984]	train-auc:0.94239	eval-auc:0.79196
[985]	train-auc:0.94250	eval-auc:0.79194
[986]	train-auc:

In [52]:
pred_train_1a

array([0.1993976 , 0.01206094, 0.04743975, ..., 0.03468203, 0.12009452,
       0.07297119], dtype=float32)

In [53]:
pred_test_1a

array([0.03523362, 0.10742633, 0.04802905, ..., 0.00910338, 0.02165737,
       0.23268175], dtype=float32)

In [54]:
pred_1a = []
for a in np.array(pred_train_1a):
    pred_1a.append(a)

len(pred_1a)

307505

In [55]:
print(type(pred_1a))

<class 'list'>


In [56]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set1x': pred_1a})
print(train_x_2.shape)
train_x_2.head()

(307505, 2)


,SK_ID_CURR,set1x
0,100002,0.199398
1,100003,0.012061
2,100004,0.047440
3,100006,0.030071
4,100007,0.023555


In [57]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set1xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [58]:
pred_1a = []
for a in np.array(pred_test_1a):
    pred_1a.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1a)

48744

In [59]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set1x': pred_1a})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [60]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set1xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [61]:
#del df
#del x_train
#del y_train
#del x_test
#del app_train
#del app_test
#del model_1b
#gc.collect()

## 特徴量生成2